In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("KafkaSparkIntegration") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1")\
    .getOrCreate()

In [8]:
kafka_broker = "172.24.16.1:9092"
kafka_topic = "customer_Churn"

df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_broker) \
    .option("subscribe", kafka_topic) \
    .load()
# Convert the binary values to string
#df = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")


In [4]:
writer = df.writeStream \
    .outputMode("append") \
    .format("memory") \
    .queryName("stock1") \
    .start()

In [13]:
df_stream = spark.sql("select * from stock1")

In [15]:
df_stock1 = spark.sql("SELECT * FROM stock1")
df_stock1.printSchema()  # Check the schema of the DataFrame
df_stock1.show(truncate=False)  # Display the data

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)

+---+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|key|value                                                                                                                                                                                                                                                                     |
+---+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1  |{"CustomerID": "651", "Age": "32", "Gender": "Female", "Tenure": "52", "Usage Frequency": "20", "S

In [17]:
schema = StructType([
    StructField("CustomerID", StringType(), True),
    StructField("Age", StringType(), True),
    StructField("Gender", StringType(), True),
    StructField("Tenure", StringType(), True),
    StructField("Usage Frequency", StringType(), True),
    StructField("Support Calls", StringType(), True),
    StructField("Payment Delay", StringType(), True),
    StructField("Subscription Type", StringType(), True),
    StructField("Contract Length", StringType(), True),
    StructField("Total Spend", StringType(), True),
    StructField("Last Interaction", StringType(), True),
    StructField("Churn", StringType(), True)
])

# Parse the JSON string in the 'value' column
df_parsed = df_stream.withColumn("parsed_json", from_json(col("value"), schema))

# Select each field from the parsed JSON
df_final = df_parsed.select(
    col("parsed_json.CustomerID").alias("CustomerID"),
    col("parsed_json.Age").alias("Age"),
    col("parsed_json.Gender").alias("Gender"),
    col("parsed_json.Tenure").alias("Tenure"),
    col("parsed_json.`Usage Frequency`").alias("UsageFrequency"),
    col("parsed_json.`Support Calls`").alias("SupportCalls"),
    col("parsed_json.`Payment Delay`").alias("PaymentDelay"),
    col("parsed_json.`Subscription Type`").alias("SubscriptionType"),
    col("parsed_json.`Contract Length`").alias("ContractLength"),
    col("parsed_json.`Total Spend`").alias("TotalSpend"),
    col("parsed_json.`Last Interaction`").alias("LastInteraction"),
    col("parsed_json.Churn").alias("Churn")
)

# Show the final DataFrame
df_final.show(truncate=False)

+----------+---+------+------+--------------+------------+------------+----------------+--------------+----------+---------------+-----+
|CustomerID|Age|Gender|Tenure|UsageFrequency|SupportCalls|PaymentDelay|SubscriptionType|ContractLength|TotalSpend|LastInteraction|Churn|
+----------+---+------+------+--------------+------------+------------+----------------+--------------+----------+---------------+-----+
|651       |32 |Female|52    |20            |2           |28          |Standard        |Quarterly     |859       |3              |0    |
|652       |56 |Female|46    |30            |0           |10          |Standard        |Annual        |924       |3              |0    |
|653       |42 |Male  |40    |12            |0           |1           |Standard        |Quarterly     |858       |26             |0    |
|654       |50 |Male  |47    |14            |9           |14          |Premium         |Monthly       |659       |26             |0    |
|655       |58 |Male  |41    |8          